# dbt Workflow Tutorial for CausaGanha

This tutorial guides you through setting up and using `dbt-duckdb` within the CausaGanha project. It is based on the plan outlined in `docs/plans/dtb.md`.

## 0. Why dbt-duckdb?

The CausaGanha project uses `dbt-duckdb` for managing its database transformations and schema. Here's why, based on the project's planning documents (`docs/plans/dtb.md`):

- **Unified Tooling**: `dbt` handles Data Definition Language (DDL), tests, lineage documentation, and CI drift-checks with a single binary.
- **Native DuckDB Support**: The `dbt-duckdb` adapter is maintained by DuckDB core developers and supports full SQL, including operations on Parquet/CSV external tables.
- **Reproducibility & Idempotency**: Models are rebuilt from source on every `dbt build`. This means development environments can be easily reset, and production builds are idempotent.
- **Built-in Data Quality**: Tests for issues like null values or non-unique entries can be defined directly alongside model code, ensuring data integrity.
- **Future-Proof**: If the project outgrows an embedded DuckDB file, the same dbt models can be used with other backends like MotherDuck or PostgreSQL with minimal changes to the dbt profile.

## 1. Installation & Setup

### 1.1 Install dbt and dbt-duckdb

If you haven't already, you'll need to install `dbt-duckdb`. This project uses `uv` for Python package management.

In [ ]:
!uv pip install "dbt-duckdb~=1.9"

### 1.2 dbt Project Structure

In CausaGanha, the dbt project is located in the `dbt/` directory at the root of the repository. If you were starting a new dbt project, you might run `dbt init your_project_name --adapter duckdb`. For this project, the structure is already established.

Let's verify the dbt project directory exists. The following command should list its contents if you are running this notebook from the `docs/tutorials/` directory.

In [ ]:
!ls ../../dbt

You should see files like `dbt_project.yml` and directories like `models/`, `seeds/`, `tests/` etc.

### 1.3 Configure `profiles.yml`

dbt uses a `profiles.yml` file to store connection configurations. This file is typically located in `~/.dbt/`. You need to ensure it's configured to point to the CausaGanha project's DuckDB database.

Here's the relevant configuration from `docs/plans/dtb.md`:

```yaml
# This content goes into your ~/.dbt/profiles.yml

causa_ganha:
  target: local
  outputs:
    local:
      type: duckdb
      path: /path/to/your/causa_ganha_repo/data/causaganha.duckdb
      # Ensure this 'path' is an absolute path or a path relative to where you run dbt commands from,
      # if you are not using the --profiles-dir option with dbt.
      # For consistency, using an absolute path is often easier for local setup.
```
**Important:** 
- Replace `/path/to/your/causa_ganha_repo/` with the actual absolute path to your cloned CausaGanha repository.
- The profile name `causa_ganha` here must match the `profile` setting in the `dbt/dbt_project.yml` file.


After configuring `~/.dbt/profiles.yml`, you can test your setup. From the root of the CausaGanha repository, run:

In [ ]:
# Make sure you are in the root of the CausaGanha repository to run this command
# If this notebook is in docs/tutorials/, you would go up two directories.
# For demonstration, we assume you'd run this in your terminal from the repo root.
!cd ../.. && dbt debug --project-dir dbt

If successful, this command will check your `profiles.yml` and `dbt_project.yml` and attempt to connect to the database. It will create `data/causaganha.duckdb` if it doesn't exist.

## 2. Repository Layout for dbt

The `docs/plans/dtb.md` plan outlines the following structure related to dbt:

```
.
├── data/                       # .duckdb file lives here, ignored by Git
│   └── causaganha.duckdb
├── dbt/                        # dbt project root
│   ├── models/
│   │   ├── staging/            # 1-to-1 raw ingests from source data
│   │   ├── marts/              # Final analytical tables (e.g., rankings)
│   │   └── seeds/              # CSV reference data loaded via dbt seed
│   ├── tests/                  # Custom SQL tests (generic tests are usually in .yml files)
│   ├── macros/                 # Custom macros 
│   └── dbt_project.yml         # Main dbt project configuration file
├── src/                        # Python application code (CLI, API, etc.)
└── .github/workflows/          # CI pipeline configurations
```
**Key points:**
- The actual DuckDB database file (`data/causaganha.duckdb`) is listed in `.gitignore` and is not committed to the repository.
- All dbt model definitions, tests, and configurations are stored within the `dbt/` directory.

## 3. Defining Models & Tests

In dbt, transformations are defined as SQL `SELECT` statements called **models**. Tests ensure data quality and integrity.

### Example Staging Model

A staging model typically performs light cleaning and preparation of raw data. For example, a model `dbt/models/staging/tjro_diary.sql` might look like this (conceptual example from `dtb.md`):

```sql
-- File: dbt/models/staging/stg_tjro_diario.sql

{{ config(materialized='table') }}

SELECT
    dj.id,
    dj.date,
    dj.url_archive,
    dj.file_hash
FROM read_parquet('path/to/source/raw/tjro_diary_*.parquet') dj -- Replace with actual source data loading, e.g. {{ source('my_source', 'tjro_raw_diaries') }}
```
**Note:** The `read_parquet` function and path are illustrative. In a real dbt project, you'd typically use `{{ source('source_name', 'table_name') }}` to refer to raw data sources defined in a `.yml` file, or `{{ ref('another_model') }}` to refer to other dbt models.

### Example Mart Model

A mart model represents data ready for analytics or reporting. For instance, `dbt/models/marts/advocate_ranking.sql` could calculate advocate rankings:

```sql
-- File: dbt/models/marts/advocate_ranking.sql

{{ config(materialized='table') }}

WITH extracted_data AS (
    SELECT
        (partes->>'advocate')::VARCHAR AS advocate, -- Example: extract advocate from JSON
        (partes->>'outcome')::VARCHAR  AS outcome   -- Example: extract outcome from JSON
    FROM {{ ref('stg_tjro_diario') }} -- Referencing the staging model
    WHERE partes IS NOT NULL -- Ensure 'partes' column is not null before trying to extract from it
)
SELECT
    advocate,
    -- elo_ranking(outcome) AS elo -- Placeholder for actual ranking logic
    COUNT(*) AS total_cases,
    SUM(CASE WHEN outcome = 'favorable' THEN 1 ELSE 0 END) AS favorable_outcomes
FROM extracted_data
WHERE advocate IS NOT NULL AND advocate <> '' -- Ensure advocate is not null or empty
GROUP BY advocate
ORDER BY favorable_outcomes DESC, total_cases DESC;
```
**Note:** The `elo_ranking(outcome)` is a placeholder for the actual OpenSkill ranking logic which might be implemented as a User-Defined Function (UDF) or a dbt macro.

### Defining Tests

dbt allows you to define tests in `.yml` files alongside your models. For example, to test the `advocate_ranking` model, you could create `dbt/models/marts/advocate_ranking.yml`:

```yaml
# File: dbt/models/marts/advocate_ranking.yml

version: 2

models:
  - name: advocate_ranking # This must match the .sql filename (advocate_ranking.sql)
    description: "Table containing advocate rankings based on case outcomes."
    columns:
      - name: advocate
        description: "The name or identifier of the advocate."
        tests:
          - not_null
          - unique
      - name: total_cases
        description: "Total number of cases for the advocate."
        tests:
          - not_null
          - dbt_utils.accepted_range:
              min_value: 0
      - name: favorable_outcomes
        description: "Number of favorable outcomes for the advocate."
        tests:
          - not_null
          - dbt_utils.accepted_range:
              min_value: 0
```
This configuration defines that the `advocate` column in the `advocate_ranking` table should not be null and should be unique. It also adds range checks for counts. dbt will automatically generate and run these tests during `dbt build` or `dbt test`.

## 4. Running dbt Locally

Once models and tests are defined, you can run dbt commands from the root of the CausaGanha repository, specifying the project directory.

### Build All Models and Run Tests
The `dbt build` command will run all your models, execute tests, and potentially build documentation and run snapshots if configured.

In [ ]:
# Ensure you are in the repo root directory for this command
!cd ../.. && dbt build --project-dir dbt

### Run Specific Models
You can run a subset of models using the `--select` flag.

In [ ]:
# Run only models in the staging directory (models within dbt/models/staging)
!cd ../.. && dbt run --select staging --project-dir dbt

### Test Specific Models
Similarly, you can test specific models.

In [ ]:
# Test the advocate_ranking model
!cd ../.. && dbt test --select advocate_ranking --project-dir dbt

### Resetting Local Database (During Alpha/Development)

The `dtb.md` plan mentions a script for easily resetting the database during development, which is useful when model structures change frequently. This might be `scripts/db/reset_dbt_database.sh`:

```bash
#!/usr/bin/env bash
# File: scripts/db/reset_dbt_database.sh (example content)
set -e

# Assuming this script is run from the repository root
DB_FILE="data/causaganha.duckdb"
DBT_PROJECT_DIR="dbt"

echo "Removing database file: $DB_FILE"
rm -f "$DB_FILE"

echo "Rebuilding dbt models..."
dbt build --project-dir "$DBT_PROJECT_DIR"

echo "✅ Fresh DB built and models materialized."
```

This script would typically be run from the repository root:

In [ ]:
# Ensure you are in the repo root directory and the script is executable
# !cd ../.. && sh ./scripts/db/reset_dbt_database.sh

Alternatively, this functionality might be integrated into the project's CLI, e.g., `uv run causaganha db reset`.

In [ ]:
# !cd ../.. && uv run causaganha db reset

## 5. CLI Integration

The `docs/plans/dtb.md` plan suggests integrating dbt commands into the existing CausaGanha CLI.

For example, the `causaganha db migrate` command, which previously might have used Alembic, could be updated to call `dbt build`:

```python
# Conceptual Python code for CLI command in src/cli.py or similar
import subprocess

def db_migrate():
    """Rebuild dbt models to the latest state."""
    try:
        # Assuming dbt is in PATH and commands are run from repo root
        subprocess.check_call(["dbt", "build", "--project-dir", "dbt"])
        print("✅ dbt models built successfully.")
    except subprocess.CalledProcessError as e:
        print(f"❌ Error during dbt build: {e}")
    except FileNotFoundError:
        print("❌ Error: dbt command not found. Is dbt installed and in your PATH?")
```
With such integration, users can continue using familiar CLI commands:

In [ ]:
# !cd ../.. && uv run causaganha db migrate

## 6. CI Pipeline (GitHub Actions)

A Continuous Integration (CI) pipeline, typically defined in `.github/workflows/`, would automate the process of building and testing dbt models on every push or pull request.

An example workflow (e.g., `.github/workflows/dbt_build_test.yml`, based on `dtb.md` Section 6) might include steps to:
1. Checkout code.
2. Set up Python and install dependencies (including `dbt-duckdb`).
3. Run `dbt build --project-dir dbt` to build models and run tests.
4. Optionally, perform a "drift check" to ensure all defined models are materialized in the database, failing the build if there are unbuilt models.

```yaml
# Example .github/workflows/dbt_build_test.yml (simplified from dtb.md)
name: CausaGanha DBT Build & Test

on: [push, pull_request]

jobs:
  build_and_test_dbt:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4

      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: "3.12" # Or your project's Python version

      - name: Install uv
        run: curl -LsSf https://astral.sh/uv/install.sh | sh

      - name: Install dbt dependencies
        run: uv pip install "dbt-duckdb~=1.9"

      - name: Run dbt build
        run: dbt build --project-dir dbt
        env:
          # CI might need a specific profiles.yml or environment variables for path configuration
          DBT_PROFILES_DIR: ./.dbt_ci # Example: use a CI-specific profiles dir

      # Optional: Add a step for schema drift check if needed
```
A `.dbt_ci/profiles.yml` might be needed, configured to use a path like `data/ci_causaganha.duckdb`.

## 7. Optional dbt Features

dbt offers several other powerful features that can be adopted as needed:

- **Seeds**: Useful for managing small, static lookup tables (e.g., country codes, status lists). Place CSV files in the `dbt/seeds/` directory and run `dbt seed --project-dir dbt` to load them into your database. These can then be referenced in models using `{{ ref('seed_file_name') }}`.

- **Snapshots**: Help track changes to mutable data over time (Slowly Changing Dimensions). For example, if an advocate's status changes, snapshots can capture the history of these changes.

- **Documentation**: dbt can automatically generate a website documenting your project, including model definitions, column descriptions, and a Directed Acyclic Graph (DAG) of model dependencies. Run `dbt docs generate --project-dir dbt` to create the documentation JSON, and then `dbt docs serve --project-dir dbt` to view it locally.

In [ ]:
# Example: Generate and serve dbt documentation (run from repo root)
# !cd ../.. && dbt docs generate --project-dir dbt
# !cd ../.. && dbt docs serve --project-dir dbt --port 8081 # Use a different port if 8080 is taken

## Conclusion

This tutorial covered the basics of setting up and using dbt with DuckDB in the CausaGanha project. Refer to the official [dbt documentation](https://docs.getdbt.com/) and the `docs/plans/dtb.md` file for more detailed information.